In [1]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_simulation_bayesimp as file

In [2]:
# Args setup
ntrial = 50
n = 500
n_int = 100
niter = 1000
front_door = True
minimise = False
add_base_kernel_BO = True

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="3:0:0",
    
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

/nfs/ghome/live/danceh/.local/mambaforge/envs/causalgp/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 11111 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41173 instead
  warnings.warn(


In [4]:
# Submitting jobs
futures = []
for seed in range(ntrial):
        f = client.submit(file.main,
                              seed,
                              n,n_int,
                              niter = niter,
                              front_door = front_door,
                              minimise = minimise,
                              add_base_kernel_BO = add_base_kernel_BO
                             )
        futures += [f]

In [11]:
futures

[<Future: finished, type: dict, key: main-ca0e3c9b6e49cb864056c0a6bf60a9f3>,
 <Future: finished, type: dict, key: main-ca0e3c9b6e49cb864056c0a6bf60a9f3>,
 <Future: finished, type: dict, key: main-6b8f14da1cbd6498e4388c1b9a7489b3>,
 <Future: finished, type: dict, key: main-f2644a9fd6207608f399cd6e09d4f3f0>,
 <Future: finished, type: dict, key: main-7b578c480925165defdb4629437b2931>,
 <Future: finished, type: dict, key: main-400de223a24f211dc016e0dc2e8e4ea9>,
 <Future: finished, type: dict, key: main-4cdd42b024f9dc7110e1a1fbd3960054>,
 <Future: finished, type: dict, key: main-654a430a516e387deed60228598816c9>,
 <Future: finished, type: dict, key: main-c126c02e5ff18849ca894f2971bd99f6>,
 <Future: finished, type: dict, key: main-566495c2659b11c1b4feedc18976452d>,
 <Future: finished, type: dict, key: main-da4c3d8fdfbf884d22613ea39b4700b6>,
 <Future: finished, type: dict, key: main-bab118031b35babf6315724537832155>,
 <Future: finished, type: dict, key: main-c6d79268578ceba5ab7a178c7838e720>,

In [12]:
# Getting results
results = client.gather(futures)

In [13]:
# Closing client
client.close()
cluster.close()

In [14]:
# Saving results
torch.save(obj = results,
           f = "simulation_bayesimp_ntrial={0}_n={1}_frontdoor={2}_minimise={3}_addbase={4}.pt".format(ntrial,
                                                                                           n,
                                                                                           front_door,
                                                                                           minimise,
                                                                                           add_base_kernel_BO)
          )